In [138]:
# Import libraries
import pgmpy.models
import pgmpy.inference
import numpy as np
from enum import Enum

In [139]:
numSlices = 24

class NodeNames(Enum):
    MITM = "MITM"
    SRM = "SRM"
    UC = "UC"
    UPS = "UPS"
    IMD = "IMD"
    MC = "MC"
    CC = "CC"

In [140]:
# Create a dynamic bayesian network
model = pgmpy.models.DynamicBayesianNetwork()
# Add nodes
model.add_nodes_from([NodeNames.MITM.value, NodeNames.SRM.value, NodeNames.UC.value, NodeNames.UPS.value])
# Add edges
for i in range(0, numSlices):
    if i != (numSlices - 1):
        for nodeName in NodeNames:
            model.add_edges_from([((nodeName.value, i), (nodeName.value, i+1))])
    
    model.add_edges_from([((NodeNames.MITM.value, i), (NodeNames.SRM.value, i)), ((NodeNames.MITM.value, i), (NodeNames.UC.value, i)),
                          ((NodeNames.SRM.value, i), (NodeNames.UPS.value, i)), ((NodeNames.UC.value, i), (NodeNames.UPS.value, i))])

# Print edges
print('--- Edges ---')
print(model.edges())
            

--- Edges ---
[(<DynamicNode(MITM, 0) at 0x7faa7b599b50>, <DynamicNode(MITM, 1) at 0x7faa7dc0cf10>), (<DynamicNode(MITM, 0) at 0x7faa7b599b50>, <DynamicNode(SRM, 0) at 0x7faa7e293a30>), (<DynamicNode(MITM, 0) at 0x7faa7b599b50>, <DynamicNode(UC, 0) at 0x7faa7e293040>), (<DynamicNode(SRM, 0) at 0x7fab681e3af0>, <DynamicNode(SRM, 1) at 0x7faa7e293670>), (<DynamicNode(SRM, 0) at 0x7fab681e3af0>, <DynamicNode(UPS, 0) at 0x7faa7e293970>), (<DynamicNode(UC, 0) at 0x7faa7af76b50>, <DynamicNode(UC, 1) at 0x7faa7e293700>), (<DynamicNode(UC, 0) at 0x7faa7af76b50>, <DynamicNode(UPS, 0) at 0x7faa7b22e5b0>), (<DynamicNode(UPS, 0) at 0x7faa7b1025b0>, <DynamicNode(UPS, 1) at 0x7faa7e293400>), (<DynamicNode(MITM, 1) at 0x7faa7dc0cf10>, <DynamicNode(SRM, 1) at 0x7faa7e293520>), (<DynamicNode(MITM, 1) at 0x7faa7dc0cf10>, <DynamicNode(UC, 1) at 0x7faa7e293910>), (<DynamicNode(SRM, 1) at 0x7faa7e293670>, <DynamicNode(UPS, 1) at 0x7faa7b22e6d0>), (<DynamicNode(UC, 1) at 0x7faa7e293700>, <DynamicNode(UPS, 1

PySmile Network

In [141]:
import pysmile
import pysmile_license
import os
import pandas as pd

In [142]:
DEBUG = False

evidenceNodes = [NodeNames.IMD.value, NodeNames.MC.value, NodeNames.CC.value]

classNodesHandles = dict()

In [156]:
def print_node_info(net, node_handle):
    print("Node id/name: " + net.get_node_id(node_handle) + "/" +
    net.get_node_name(node_handle))
    print(" Outcomes: " + " ".join(net.get_outcome_ids(node_handle)))
    parent_ids = net.get_parent_ids(node_handle)
    if len(parent_ids) > 0:
        print(" Parents: " + " ".join(parent_ids))
    child_ids = net.get_child_ids(node_handle)
    if len(child_ids) > 0:
        print(" Children: " + " ".join(child_ids))
    print_cpt_matrix(net, node_handle)
    
def print_cpt_matrix(net, node_handle):
    cpt = net.get_node_definition(node_handle)
    parents = net.get_parents(node_handle)
    dim_count = 1 + len(parents)
    dim_sizes = [0] * dim_count
    for i in range(0, dim_count - 1):
        dim_sizes[i] = net.get_outcome_count(parents[i])
    dim_sizes[len(dim_sizes) - 1] = net.get_outcome_count(node_handle)
    coords = [0] * dim_count
    for elem_idx in range(0, len(cpt)):
        index_to_coords(elem_idx, dim_sizes, coords)
        outcome = net.get_outcome_id(node_handle, coords[dim_count - 1])
        out_str = " P(" + outcome
        if dim_count > 1:
            out_str += " | "
            for parent_idx in range(0, len(parents)):
                if parent_idx > 0:
                    out_str += ","
                parent_handle = parents[parent_idx]
                out_str += net.get_node_id(parent_handle) + "=" + \
                net.get_outcome_id(parent_handle, coords[parent_idx])
        prob = cpt[elem_idx]
        out_str += ")=" + str(prob)
        print(out_str)
    
        
def index_to_coords(index, dim_sizes, coords):
    prod = 1
    for i in range(len(dim_sizes) - 1, -1, -1):
        coords[i] = int(index / prod) % dim_sizes[i]
        prod *= dim_sizes[i]

def pint_time_cpt_marix(net, nodeHandle):
    timeCPT = net.get_node_temporal_definition(nodeHandle, 1)
    
        
def plot_time_CPT(net, nodeHandle):    
    cpt = net.get_node_temporal_definition(nodeHandle, 1)
    print(len(cpt))
    print("###")
    
def print_net_info(net, unrolled = True):
    for n in net.get_all_nodes():
        print_node_info(net, n)
        if not unrolled and net.get_node_id(n) == NodeNames.UPS.value:
            plot_time_CPT(net, n)
            
def calc_precision(confMatrix, outcome):
    TP = confMatrix[outcome][outcome]
    FP = 0
    for i in range(0, len(confMatrix[outcome])):
        if i != outcome:
            FP += confMatrix[outcome][i]
    if TP + FP == 0: return float('nan')
    return TP / (TP + FP)
            
def print_validator_results(net, originalSliceCount, validator, nodeId): 
    for slice in range(1, originalSliceCount):
        print("### Slice " + str(slice) + " ###")
        nodeHandle = classNodesHandles[nodeId + "_" + str(slice)]
        outcomeCount = net.get_outcome_count(nodeHandle)
        cm = validator.get_confusion_matrix(nodeHandle)
        for i in range(0, outcomeCount):
            acc = validator.get_accuracy(nodeHandle, i)
            print("Accuracy for " + nodeId + str(i) + ": " + str(acc))
            #prec = calc_precision(cm, i)
            #print("Precision for " + nodeId + str(i) + ": " + str(prec))
        print("** Confusion Matrix **")
        for i in range(0, outcomeCount):
            print(cm[i])
        print("")

In [159]:
# Create and read the DBN from file
net = pysmile.Network()
ds = pysmile.learning.DataSet()

net.read_file(os.getcwd() + "/../../../Genie-DBN/DBN-MITM.xdsl")
df = pd.read_csv(os.getcwd() + "/outTrain.csv")
ds.read_pandas_dataframe(df)
   
unrolledNet = net.unroll().unrolled     
matching = ds.match_network(unrolledNet)
validator = pysmile.learning.Validator(unrolledNet, ds, matching)
# Set class nodes (those that will not be considered as evidence nodes)
for elem in NodeNames.__members__:
    if elem not in evidenceNodes:
        classNodesHandles[elem] = unrolledNet.get_node(elem)
        validator.add_class_node(classNodesHandles[elem])
        for slice in range(1, net.get_slice_count()):
            elemCat = elem + "_" + str(slice)
            classNodesHandles[elemCat] = unrolledNet.get_node(elemCat)
            validator.add_class_node(classNodesHandles[elemCat])
# Test the predctions on the class nodes            
validator.test()
validator.get_result_data_set()
print_validator_results(unrolledNet, net.get_slice_count(), validator, NodeNames.UPS.value)


if DEBUG:
    print_net_info(unrolledNet, unrolled=True)


    

### Slice 1 ###
Accuracy for UPS0: 1.0
Precision for UPS0: 1.0
Accuracy for UPS1: nan
Precision for UPS1: nan
** Confusion Matrix **
[50, 0]
[0, 0]

### Slice 2 ###
Accuracy for UPS0: 1.0
Precision for UPS0: 1.0
Accuracy for UPS1: nan
Precision for UPS1: nan
** Confusion Matrix **
[50, 0]
[0, 0]

### Slice 3 ###
Accuracy for UPS0: 1.0
Precision for UPS0: 1.0
Accuracy for UPS1: 0.0
Precision for UPS1: 0.0
** Confusion Matrix **
[48, 0]
[2, 0]

### Slice 4 ###
Accuracy for UPS0: 1.0
Precision for UPS0: 1.0
Accuracy for UPS1: 0.0
Precision for UPS1: 0.0
** Confusion Matrix **
[48, 0]
[2, 0]

### Slice 5 ###
Accuracy for UPS0: 1.0
Precision for UPS0: 1.0
Accuracy for UPS1: 0.0
Precision for UPS1: 0.0
** Confusion Matrix **
[48, 0]
[2, 0]

### Slice 6 ###
Accuracy for UPS0: 1.0
Precision for UPS0: 1.0
Accuracy for UPS1: 0.0
Precision for UPS1: 0.0
** Confusion Matrix **
[47, 0]
[3, 0]

### Slice 7 ###
Accuracy for UPS0: 1.0
Precision for UPS0: 1.0
Accuracy for UPS1: 0.0
Precision for UPS1: 